# RoadSegment — Length Analysis & Report

This notebook focuses **only** on the `RoadSegment` class:
1. **Query** geometry as WKT and/or GeoJSON from your SPARQL endpoint (Fuseki or proxy).
2. **Parse & project** to a metric CRS (UTM) and compute per‑segment **length (meters)**.
3. **Validate** with an optional geodesic check.
4. **Generate** a concise PDF report and CSV export.


## Prerequisites

Install once:
```bash
pip install requests pandas shapely pyproj matplotlib
```


In [1]:

# ==== CONFIG ====
# Use ONE endpoint:
# A) Direct Fuseki SPARQL:
ENDPOINT = "https://a19b8c2c5b14.ngrok-free.app/amaravati/sparql"
# B) Or FastAPI proxy (JSON):
# ENDPOINT = "http://localhost:8111/query"

USE_PROXY_JSON = ENDPOINT.endswith("/query")

# Stable UTM for India (meters). Set None to auto-pick from centroids.
DEFAULT_EPSG = 32644

# Output paths
from pathlib import Path
OUT_DIR = Path.cwd() / "outputs"
OUT_DIR.mkdir(parents=True, exist_ok=True)
PDF_PATH = OUT_DIR / "roadsegment_length_report.pdf"
CSV_PATH = OUT_DIR / "roadsegment_lengths.csv"


In [2]:

import json, re, math
import pandas as pd
import requests
from shapely import wkt as shapely_wkt
from shapely.geometry import shape as shapely_shape, LineString, MultiLineString, Polygon, MultiPolygon
from shapely.ops import transform as shp_transform
from pyproj import CRS, Transformer, Geod
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

PREFIXES = """
PREFIX adto: <http://www.projectsynapse.com/ontologies/adto#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX geo:  <http://www.opengis.net/ont/geosparql#>
"""


In [4]:

def sparql_select(query: str) -> pd.DataFrame:
    if USE_PROXY_JSON:
        r = requests.post(
            ENDPOINT,
            headers={"Content-Type": "application/json"},
            json={"query": PREFIXES + query},
            timeout=120,
        )
    else:
        r = requests.post(
            ENDPOINT,
            headers={"Accept":"application/sparql-results+json",
                     "Content-Type":"application/sparql-query"},
            data=(PREFIXES + query).encode("utf-8"),
            timeout=120,
        )
    r.raise_for_status()
    data = r.json()
    rows = [{k:v.get("value") for k,v in b.items()} for b in data.get("results",{}).get("bindings",[])]
    return pd.DataFrame(rows)


In [5]:

def fetch_roadsegments() -> pd.DataFrame:
    q = """
    SELECT ?s ?name ?status
           (COALESCE(?wkt_geo, ?wkt_adto)    AS ?wkt)
           (COALESCE(?gj_geom, ?gj_subject)  AS ?geojson)
    WHERE {
      ?s a adto:RoadSegment .
      OPTIONAL { ?s adto:hasName ?name }
      OPTIONAL { ?s adto:hasStatus ?status }

      OPTIONAL {
        ?s (geo:hasGeometry|adto:hasGeometry) ?g .
        OPTIONAL { ?g geo:asWKT  ?wkt_geo }
        OPTIONAL { ?g adto:asWKT ?wkt_adto }
      }
      OPTIONAL {
        ?s (geo:hasGeometry|adto:hasGeometry) ?g2 .
        OPTIONAL { ?g2 adto:asGeoJSON ?gj_geom }   # adjust if your data uses a different predicate
      }
      OPTIONAL { ?s adto:asGeoJSON ?gj_subject }
    }
    ORDER BY ?s
    """
    df = sparql_select(q)
    if not df.empty:
        df = df.rename(columns={"s":"iri"})
    for col in ("iri","name","status","wkt","geojson"):
        if col not in df.columns:
            df[col] = pd.NA
    return df


In [6]:

def _clean_wkt(s: str) -> str:
    if not s:
        return s
    s2 = str(s).strip()
    if s2.upper().startswith("SRID=") and ";" in s2:
        s2 = s2.split(";",1)[1].strip()
    m = re.match(r"^<[^>]+>\s*(.*)$", s2)  # drop '<...CRS...>' prefix
    if m: s2 = m.group(1).strip()
    return s2

def _geom_from_literals(wkt_str: str | None, geojson_str: str | None):
    if wkt_str:
        try:
            return shapely_wkt.loads(_clean_wkt(wkt_str))
        except Exception:
            pass
    if geojson_str:
        try:
            gj = json.loads(geojson_str)
            if isinstance(gj, dict) and gj.get("type") == "Feature":
                gj = gj.get("geometry")
            return shapely_shape(gj) if gj else None
        except Exception:
            pass
    return None

def _utm_epsg_for_lonlat(lon: float, lat: float) -> int:
    zone = int(math.floor((lon + 180) / 6) + 1)
    return 32600 + zone if lat >= 0 else 32700 + zone

def project_to_meters(df: pd.DataFrame, default_epsg: int | None = DEFAULT_EPSG):
    df = df.copy()
    for col in ("wkt","geojson"):
        if col not in df.columns:
            df[col] = pd.Series([None]*len(df), dtype="object")
    df["geom"] = [ _geom_from_literals(w,g) for w,g in zip(df["wkt"], df["geojson"]) ]
    df["geom_m"] = pd.Series([None]*len(df), dtype="object")

    valid = df["geom"].dropna()
    if valid.empty:
        return df, CRS.from_epsg(default_epsg or 4326)

    lons = [g.centroid.x for g in valid]
    lats = [g.centroid.y for g in valid]
    mean_lon = sum(lons)/len(lons)
    mean_lat = sum(lats)/len(lats)

    epsg = default_epsg or _utm_epsg_for_lonlat(mean_lon, mean_lat)
    crs_m = CRS.from_epsg(epsg)

    transformer = Transformer.from_crs("EPSG:4326", crs_m, always_xy=True)
    def _proj(g):
        return shp_transform(lambda x,y, z=None: transformer.transform(x,y), g)

    df.loc[valid.index, "geom_m"] = [ _proj(g) for g in valid ]
    return df, crs_m

def length_m(g):
    if g is None:
        return 0.0
    if isinstance(g, (LineString, MultiLineString)):
        return g.length
    if isinstance(g, (Polygon, MultiPolygon)):
        return g.length
    return 0.0


In [7]:

# Fetch
roads = fetch_roadsegments()
print("Rows fetched:", len(roads))
print(roads.head(3)[["iri","name","status","wkt","geojson"]])

# Project & compute lengths
roads_p, crs_m = project_to_meters(roads)
roads_p["length_m"] = roads_p["geom_m"].map(length_m)

print("\nCRS used:", crs_m.to_string())
print("Total segments:", len(roads_p))
print("Total length (m):", roads_p["length_m"].sum())

# Export CSV
roads_p[["iri","name","status","length_m"]].to_csv(CSV_PATH, index=False)
print("Saved CSV:", CSV_PATH)


Rows fetched: 95
                                                 iri       name       status  \
0  http://www.projectsynapse.com/ontologies/adto#...  E10_32_01    Completed   
1  http://www.projectsynapse.com/ontologies/adto#...  E10_32_02  Maintenance   
2  http://www.projectsynapse.com/ontologies/adto#...  E11_33_01    Completed   

                                                 wkt  \
0  <http://www.opengis.net/def/crs/OGC/1.3/CRS84>...   
1  <http://www.opengis.net/def/crs/OGC/1.3/CRS84>...   
2  <http://www.opengis.net/def/crs/OGC/1.3/CRS84>...   

                                             geojson  
0  {"type":"LineString","coordinates":[[80.486047...  
1  {"type":"LineString","coordinates":[[80.514753...  
2  {"type":"LineString","coordinates":[[80.503895...  

CRS used: EPSG:32644
Total segments: 95
Total length (m): 325901.2443941693
Saved CSV: f:\Projects\Hackthon\AURA\AURA\outputs\roadsegment_lengths.csv


In [8]:

# Optional geodesic check
geod = Geod(ellps="WGS84")

def geodesic_len_m(geom_ll):
    if geom_ll is None:
        return 0.0
    if isinstance(geom_ll, LineString):
        xs, ys = zip(*list(geom_ll.coords))
        return geod.line_length(xs, ys)
    if isinstance(geom_ll, MultiLineString):
        return sum(geodesic_len_m(g) for g in geom_ll.geoms)
    return 0.0

sample = roads_p.sample(min(20, len(roads_p)), random_state=1) if len(roads_p) else roads_p.head(0)
sample = sample.copy()
sample["length_geod_m"] = sample["geom"].map(geodesic_len_m)
sample["ratio_proj_vs_geod"] = (sample["length_m"] / sample["length_geod_m"]).replace([float("inf")], None)
print("\nGeodesic check on sample:")
print(sample[["iri","length_m","length_geod_m","ratio_proj_vs_geod"]].head(10).to_string(index=False))



Geodesic check on sample:
                                                    iri    length_m  length_geod_m  ratio_proj_vs_geod
 http://www.projectsynapse.com/ontologies/adto#E8_30_02 6850.180147    6852.694454            0.999633
 http://www.projectsynapse.com/ontologies/adto#E5_27_01 5172.101469    5173.959066            0.999641
 http://www.projectsynapse.com/ontologies/adto#N11_8_02 4137.509216    4138.997634            0.999640
 http://www.projectsynapse.com/ontologies/adto#N17_2_02  615.498930     615.717020            0.999646
http://www.projectsynapse.com/ontologies/adto#N4B_22_02 4070.653664    4072.157721            0.999631
 http://www.projectsynapse.com/ontologies/adto#N13_6_01 4247.196499    4248.719140            0.999642
 http://www.projectsynapse.com/ontologies/adto#N8_11_02 4297.069138    4298.627924            0.999637
 http://www.projectsynapse.com/ontologies/adto#N10_9_02 1940.519769    1941.219253            0.999640
  http://www.projectsynapse.com/ontologies/adt

In [9]:

# PDF report (summary + histogram + top segments + map)
with PdfPages(str(PDF_PATH)) as pdf:
    # Page 1 — Summary
    fig = plt.figure(figsize=(8.5, 11)); ax = fig.add_subplot(111); ax.axis("off")
    ax.text(0.02, 0.95, "RoadSegment Length Report", fontsize=18, weight="bold")
    ax.text(0.02, 0.91, f"Projected CRS: {crs_m.to_string()}", fontsize=10)
    ax.text(0.02, 0.88, f"Segments: {len(roads_p)}", fontsize=10)
    ax.text(0.02, 0.85, f"Total length: {roads_p['length_m'].sum():,.0f} m", fontsize=10)
    pdf.savefig(fig); plt.close(fig)

    # Page 2 — Histogram
    fig = plt.figure(figsize=(8.5, 11))
    ax = fig.add_subplot(111)
    ax.set_title("Road segment lengths (m)")
    roads_p["length_m"].plot(kind="hist", bins=24, ax=ax)
    ax.set_xlabel("meters"); ax.set_ylabel("count")
    pdf.savefig(fig); plt.close(fig)

    # Page 3 — Top segments table
    fig = plt.figure(figsize=(8.5, 11)); ax = fig.add_subplot(111); ax.axis("off")
    ax.set_title("Top segments by length (m)")
    top = roads_p[["iri","name","status","length_m"]].sort_values("length_m", ascending=False).head(20).copy()
    top["length_m"] = top["length_m"].map(lambda v: f"{v:,.0f}")
    table = ax.table(cellText=top.values, colLabels=list(top.columns), loc="upper left", colLoc="left", cellLoc="left")
    table.auto_set_font_size(False); table.set_fontsize(8); table.scale(1, 1.2)
    pdf.savefig(fig); plt.close(fig)

    # Page 4 — Quick map
    fig = plt.figure(figsize=(8.5, 11))
    ax = fig.add_subplot(111)
    ax.set_title("Road segments (projected meters)")
    for g in roads_p["geom_m"].dropna().head(1000):
        try:
            if isinstance(g, LineString):
                x, y = g.xy; ax.plot(x, y, linewidth=0.6)
            elif isinstance(g, MultiLineString):
                for part in g.geoms:
                    x, y = part.xy; ax.plot(x, y, linewidth=0.6)
        except Exception:
            pass
    ax.set_xlabel("X (m)"); ax.set_ylabel("Y (m)")
    pdf.savefig(fig); plt.close(fig)

print("Saved PDF:", PDF_PATH)


Saved PDF: f:\Projects\Hackthon\AURA\AURA\outputs\roadsegment_length_report.pdf
